In [1]:
import datetime

from psycopg2.extras import DictCursor

from etl_utils.pipeline import start_etl_process
from common.settings import (
    CONNECTION_CONFIG,
    INDEX_NAME,
)
from common.queries import QUERY_FILM_WORK

In [2]:
from etl_utils.extraction import connect_to_postgresql

In [3]:
CONNECTION_CONFIG["host"] = "127.0.0.1"
ELASTIC_HOST = "http://localhost:9200/"

In [4]:
start_etl_process(
    CONNECTION_CONFIG,
    ELASTIC_HOST,
    "storage.json",
    QUERY_FILM_WORK,
    INDEX_NAME
)

timestamp=2024-04-25 19:24:28,014 | logger_name=storage | level=ERROR | line=34 | message=exception during state loading: [Errno 2] No such file or directory: 'storage.json';
timestamp=2024-04-25 19:24:28,045 | logger_name=etl | level=DEBUG | line=64 | message=save date into storage: 1895-03-21 00:00:00+00:00;
timestamp=2024-04-25 19:24:28,575 | logger_name=etl | level=DEBUG | line=57 | message=process 100 rows;
timestamp=2024-04-25 19:24:28,745 | logger_name=etl | level=DEBUG | line=57 | message=process 100 rows;
timestamp=2024-04-25 19:24:28,891 | logger_name=etl | level=DEBUG | line=57 | message=process 100 rows;
timestamp=2024-04-25 19:24:29,022 | logger_name=etl | level=DEBUG | line=57 | message=process 100 rows;
timestamp=2024-04-25 19:24:29,143 | logger_name=etl | level=DEBUG | line=57 | message=process 100 rows;
timestamp=2024-04-25 19:24:29,280 | logger_name=etl | level=DEBUG | line=57 | message=process 100 rows;
timestamp=2024-04-25 19:24:29,388 | logger_name=etl | level=DEBU

KeyboardInterrupt: 

In [37]:
query_persons = """
SELECT
   fw.id,
   fw.title,
   fw.description,
   fw.rating,
   fw.type,
   MAX(g.updated_at) updated_at,
   COALESCE (
       json_agg(
           DISTINCT jsonb_build_object(
               'person_role', pfw.role,
               'person_id', p.id,
               'person_name', p.full_name
           )
       ) FILTER (WHERE p.id is not null),
       '[]'
   ) as persons,
   array_agg(DISTINCT g.name) as genres
FROM genre g
LEFT JOIN genre_film_work gfw ON g.id = gfw.genre_id
LEFT JOIN film_work fw ON gfw.film_work_id = fw.id
LEFT JOIN person_film_work pfw ON pfw.film_work_id = fw.id
LEFT JOIN person p ON pfw.person_id = p.id
WHERE g.updated_at > %s
GROUP BY fw.id
ORDER BY updated_at;
"""

In [38]:
date = datetime.datetime(
    year=1800,
    month=1,
    day=1,
)

In [39]:
with connect_to_postgresql(CONNECTION_CONFIG) as connection:
    cursor = connection.cursor(cursor_factory=DictCursor)
    cursor.execute(query_persons, (str(date), ))

    data = cursor.fetchall()

In [40]:
len(data)

999

In [41]:
from common.settings import ELASTIC_HOST